In [1]:
import json
from config import (train_data_path, 
                    test_data_path)

def json_dataloader(train_fd, test_fd):

    #defining train and test data 

    train_data = json.load(train_fd)
    test_data = json.load(test_fd)

    return (train_data, 
            test_data)

#loading file descriptors for train and test data

train_fd = open(train_data_path)
test_fd = open(test_data_path)

train_data, test_data = json_dataloader(train_fd, test_fd)

In [4]:
print(train_data['1909.00694'].keys())

dict_keys(['title', 'abstract', 'full_text', 'qas', 'figures_and_tables'])


In [2]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)
squad['train'][0]

Found cached dataset squad (C:/Users/DELL/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


{'id': '56d538bc2593cc1400307aed',
 'title': '2008_Sichuan_earthquake',
 'context': 'Reginald DesRoches, a professor of civil and environmental engineering at Georgia Tech, pointed out that the massive damage of properties and houses in the earthquake area was because China did not create an adequate seismic design code until after the devastating 1976 Tangshan earthquake. DesRoches said: "If the buildings were older and built prior to that 1976 earthquake, chances are they weren\'t built for adequate earthquake forces."',
 'question': 'What is the profession of Reginald DesRoches?',
 'answers': {'text': ['professor of civil and environmental engineering'],
  'answer_start': [22]}}

In [31]:
squad['train'][1242]

{'id': '56cfbc81234ae51400d9bf3b',
 'title': 'New_York_City',
 'context': 'Several prominent American literary figures lived in New York during the 1830s and 1840s, including William Cullen Bryant, Washington Irving, Herman Melville, Rufus Wilmot Griswold, John Keese, Nathaniel Parker Willis, and Edgar Allan Poe. Public-minded members of the contemporaneous business elite lobbied for the establishment of Central Park, which in 1857 became the first landscaped park in an American city.',
 'question': 'Central park, in 1857, became the first park in America to become what?',
 'answers': {'text': ['first landscaped'], 'answer_start': [372]}}

In [3]:
qasper = load_dataset("allenai/qasper")
# qasper = qasper.train_test_split(test_size=0.3)

Found cached dataset qasper (C:/Users/DELL/.cache/huggingface/datasets/allenai___qasper/qasper/0.3.0/2bfcd239e581ab83f9ab7b76a82e42c6bcf574a13246ae6cc5a6c357c35f96f9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
print(qasper)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstract', 'full_text', 'qas', 'figures_and_tables'],
        num_rows: 888
    })
    validation: Dataset({
        features: ['id', 'title', 'abstract', 'full_text', 'qas', 'figures_and_tables'],
        num_rows: 281
    })
    test: Dataset({
        features: ['id', 'title', 'abstract', 'full_text', 'qas', 'figures_and_tables'],
        num_rows: 416
    })
})


In [20]:
qasper['train']

Dataset({
    features: ['id', 'title', 'abstract', 'full_text', 'qas', 'figures_and_tables'],
    num_rows: 888
})

In [26]:
print(qasper['train'][0]['id'])
print(qasper['train'][0]['title'])
print(qasper['train'][0]['abstract'])
print(qasper['train'][0]['full_text'])
print(qasper['train'][0]['qas'])
print(qasper['train'][0]['figures_and_tables'])

1909.00694
Minimally Supervised Learning of Affective Events Using Discourse Relations
Recognizing affective events that trigger positive or negative sentiment has a wide range of natural language processing applications but remains a challenging problem mainly because the polarity of an event is not necessarily predictable from its constituent words. In this paper, we propose to propagate affective polarity using discourse relations. Our method is simple and only requires a very small seed lexicon and a large raw corpus. Our experiments using Japanese data show that our method learns affective events effectively without manually labeled data. It also improves supervised learning results when labeled data are small.
{'section_name': ['Introduction', 'Related Work', 'Proposed Method', 'Proposed Method ::: Polarity Function', 'Proposed Method ::: Discourse Relation-Based Event Pairs', 'Proposed Method ::: Discourse Relation-Based Event Pairs ::: AL (Automatically Labeled Pairs)', 'Propos

In [27]:
print(qasper['train'][0]['qas'].keys())

dict_keys(['question', 'question_id', 'nlp_background', 'topic_background', 'paper_read', 'search_query', 'question_writer', 'answers'])


In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [33]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [37]:
print(tokenized_squad['train'][0])

{'input_ids': [101, 1327, 3515, 14703, 17038, 3482, 1108, 1215, 1107, 1103, 3838, 136, 102, 1130, 1382, 117, 24041, 1373, 1114, 170, 2783, 1104, 1168, 13073, 12854, 1146, 1105, 1666, 170, 1888, 2322, 1111, 107, 3177, 20993, 138, 7382, 107, 117, 170, 16516, 17482, 6620, 1389, 3098, 1118, 170, 1372, 1104, 22173, 1646, 4398, 1116, 1105, 1639, 2011, 1106, 2933, 1103, 2877, 1433, 1154, 1231, 24365, 4419, 1157, 2560, 1654, 3892, 117, 1378, 1103, 9908, 15867, 5395, 1323, 4598, 119, 24041, 1245, 1126, 9088, 1111, 1103, 1368, 1291, 4243, 12403, 2295, 2322, 1274, 3798, 1123, 1461, 107, 146, 3982, 3446, 107, 1105, 1157, 1390, 1888, 117, 2046, 1107, 1103, 7414, 117, 1106, 1103, 2322, 119, 1130, 1381, 117, 1122, 1108, 1717, 1115, 24041, 1156, 1250, 1114, 18613, 1918, 16164, 4820, 1105, 13359, 6859, 144, 1811, 10430, 1182, 1113, 170, 144, 15072, 107, 11318, 3263, 1111, 9091, 107, 2322, 1115, 8469, 1106, 2819, 2130, 9712, 25603, 119, 1109, 2322, 117, 1134, 4086, 1113, 1428, 1743, 117, 1108, 1383, 110